<a href="https://colab.research.google.com/github/andresbadillo/aenc_tfroc/blob/master/aenc_tfroc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Consumo = aenc / Factor de pérdidas del `-día-`

En este pieza se código se cargan los archivos aenc y tfroc del mismo día seleccionado y se realiza la división de cada valor horario por el factor de perdidas

In [ ]:
import pandas as pd

# Cargar los archivos desde Google Colab
from google.colab import files

# Subir los archivos
print("Por favor, sube el archivo aenc")
aenc_file = files.upload()

print("\nPor favor, sube el archivo tfroc")
tfroc_file = files.upload()

# Leer los archivos con el separador ";"
aenc_file_path = list(aenc_file.keys())[0]
tfroc_file_path = list(tfroc_file.keys())[0]

aenc_df = pd.read_csv(aenc_file_path, sep=";", encoding='latin1')
tfroc_df = pd.read_csv(tfroc_file_path, sep=";", encoding='latin1')

# Renombrar columnas para facilitar la unión
aenc_df.rename(columns={"CODIGO SIC": "CODIGO FRONTERA"}, inplace=True)

# Unir los dos dataframes utilizando "CODIGO FRONTERA"
merged_df = pd.merge(aenc_df, tfroc_df[["CODIGO FRONTERA", "FACTOR DE PERDIDAS"]], on="CODIGO FRONTERA", how="inner")

# Dividir los valores de aenc horario por el FACTOR DE PERDIDAS
horas = [col for col in aenc_df.columns if "HORA" in col]
for hora in horas:
    merged_df[hora] = merged_df[hora] / merged_df["FACTOR DE PERDIDAS"]

# Agregar columnas adicionales al archivo de salida
merged_df.rename(columns={"CODIGO PROPIO": "NOMBRE FRONTERA"}, inplace=True)
output_df = merged_df[["CODIGO FRONTERA", "NOMBRE FRONTERA", "TIPO DE AGRUPACIÓN", "IMPO - EXPO"] + horas]

# Agregar columna de total de consumo por frontera
output_df["TOTAL CONSUMO"] = output_df[horas].sum(axis=1)

# Exportar a un archivo CSV
output_file_name = "consumos_1201.csv"
output_df.to_csv(output_file_name, index=False, sep=";")

# Descargar el archivo resultante
files.download(output_file_name)
print(f"\nEl archivo {output_file_name} ha sido descargado")

## Consumo = aenc / Factor de pérdidas del `-Mes-`

En este pieza se código se cargan los archivos aenc y tfroc del mismo mes y se realiza la división de cada valor horario por el factor de perdidas. Hay dos archivos exportados:
- Archivo que consolida todos los aenc del mes: aenc_consolidado_`{mm}`_`{aaaa}`.csv
- Archivo que consolida los consumos del mes: consumos_`{mm}`_`{aaaa}`.csv

In [ ]:
!pip install holidays_co

import pandas as pd
import os
import time
from datetime import datetime
import holidays_co

# Configurar el año
anio_actual = 2025  # Cambiar este valor según sea necesario

# Cargar los archivos desde Google Colab
from google.colab import files

# Subir los archivos
aenc_files = []
tfroc_files = []

print("Por favor, sube todos los archivos de 'aenc' del mes")
aenc_uploaded = files.upload()
aenc_files.extend(aenc_uploaded.keys())

print("\nPor favor, sube todos los archivos de 'tfroc' del mes")
tfroc_uploaded = files.upload()
tfroc_files.extend(tfroc_uploaded.keys())

# Inicializar dataframes vacíos para consolidar los resultados
consolidated_aenc_df = pd.DataFrame()
consolidated_df = pd.DataFrame()

# Función para obtener el nombre del día en español
def obtener_dia_semana(fecha):
    dias_semana = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
    dia_semana = datetime.strptime(fecha, "%d-%m-%Y").weekday()
    return dias_semana[dia_semana]

# Obtener los festivos de Colombia para el año configurado
festivos = holidays_co.get_colombia_holidays_by_year(anio_actual)
#print(f"Festivos del año {anio_actual}: {festivos}")

# Función para determinar el tipo de día
def tipo_dia(fecha):
    fecha_obj = datetime.strptime(fecha, "%d-%m-%Y").date()

    if holidays_co.is_holiday_date(fecha_obj):  # Verificar si es festivo
        return "Festivo"

    # Si no es festivo, determinar si es lunes a viernes, sábado o domingo
    dia_semana = fecha_obj.weekday()  # 0: Lunes, 1: Martes, ..., 6: Domingo
    if dia_semana == 5:  # Sábado
        return "Sábado"
    elif dia_semana == 6:  # Domingo
        return "Domingo"
    else:  # Lunes a Viernes
        return "Hábil"

# Diccionario de mapeo para la columna "OR"
mapeo_or = {
    "CUNM": "ENEL",
    "SOLM": "AIRE",
    "SANM": "ESSA",
    "MARM": "AFINIA",
    "NSAM": "ESSA",
    "BOYM": "EBSA",
    "CASM": "ENERCA",
    "METM": "EMSA",
    "QUIM": "EDEQ",
    "RUIM": "RUITOQUE",
    "CHOM": "DISPAC",
    "CLOM": "EMCALI"
}

# Función para mapear el NT por rangos
def mapear_nt_por_rango(valor):
    if valor in {1, 2}:  # Comprobamos si el valor es exactamente 1 o 2
        return valor
    elif valor < 1:
        return 1
    elif valor < 30:
        return 2
    elif valor < 57.5:
        return 3
    return None  # O un valor por defecto si no encaja en ningún rango

# Procesar archivos día a día
for aenc_file_path in aenc_files:
    # Extraer el día y el mes del nombre del archivo
    day = os.path.splitext(aenc_file_path)[0][-4:]  # Los últimos 4 caracteres son el día
    date = f"{day[2:]}-{day[:2]}-{anio_actual}"  # Formatear como dd-mm-aaaa

    # Buscar el archivo tfroc correspondiente
    tfroc_file_path = next((f for f in tfroc_files if day in f), None)
    if tfroc_file_path is None:
        print(f"No se encontró un archivo tfroc correspondiente para el día {day}.")
        continue

    # Leer los archivos con el separador ";"
    aenc_df = pd.read_csv(aenc_file_path, sep=";", encoding='latin1')
    tfroc_df = pd.read_csv(tfroc_file_path, sep=";", encoding='latin1')

    # Renombrar columnas para facilitar la unión
    aenc_df.rename(columns={"CODIGO SIC": "CODIGO FRONTERA"}, inplace=True)

    # Agregar columnas adicionales al dataframe de aenc
    horas = [col for col in aenc_df.columns if "HORA" in col]
    aenc_df.insert(0, "FECHA", date)
    aenc_df.insert(1, "DIA", obtener_dia_semana(date))
    aenc_df.insert(2, "TIPO_DIA", tipo_dia(date))
    aenc_df["TOTAL CONSUMO"] = aenc_df.loc[:, horas].sum(axis=1)

    # Consolidar los datos de aenc
    consolidated_aenc_df = pd.concat([consolidated_aenc_df, aenc_df], ignore_index=True)

    # Unir los dos dataframes utilizando "CODIGO FRONTERA"
    merged_df = pd.merge(aenc_df, tfroc_df[["CODIGO FRONTERA", "FACTOR DE PERDIDAS", "MERCADO COMERCIALIZACIÓN QUE EXPORTA", "NIVEL DE TENSION"]], on="CODIGO FRONTERA", how="inner")

    # Dividir los valores de consumo horario por el FACTOR DE PERDIDAS
    for hora in horas:
        merged_df[hora] = merged_df[hora] / merged_df["FACTOR DE PERDIDAS"]

    # Agregar columnas adicionales al archivo de salida
    merged_df.rename(columns={"CODIGO PROPIO": "NOMBRE FRONTERA"}, inplace=True)
    daily_df = merged_df.loc[:, ["CODIGO FRONTERA", "NOMBRE FRONTERA", "MERCADO COMERCIALIZACIÓN QUE EXPORTA", "NIVEL DE TENSION","TIPO DE AGRUPACIÓN", "IMPO - EXPO"] + horas]

    # Agregar columna de fecha, día y tipo de día
    daily_df.insert(0, "FECHA", date)
    daily_df.insert(1, "DIA", obtener_dia_semana(date))
    daily_df.insert(2, "TIPO_DIA", tipo_dia(date))

    # Agregar columna "OR" basada en el mapeo
    daily_df.insert(6, "OR", daily_df["MERCADO COMERCIALIZACIÓN QUE EXPORTA"].map(mapeo_or))

    # Agregar columna "NT" basada en la función de mapeo por rangos
    daily_df.insert(8, "NT", daily_df["NIVEL DE TENSION"].map(mapear_nt_por_rango))

    # Agregar columna de total de consumo por frontera
    daily_df["TOTAL CONSUMO"] = daily_df.loc[:, horas].sum(axis=1)

    # Agregar los resultados al dataframe consolidado
    consolidated_df = pd.concat([consolidated_df, daily_df], ignore_index=True)

# Exportar el archivo de aenc consolidado
mes_archivos = aenc_files[0][4:6]  # Obtener el mes de los archivos cargados
output_aenc_file_name = f"aenc_consolidado_{mes_archivos}_{anio_actual}.csv"
consolidated_aenc_df.to_csv(output_aenc_file_name, index=False, sep=";", encoding='utf-8-sig')
files.download(output_aenc_file_name)
print(f"\nEl archivo {output_aenc_file_name} ha sido descargado")

# Exportar el archivo de consumos consolidado
output_file_name = f"consumos_{mes_archivos}_{anio_actual}.csv"
consolidated_df.to_csv(output_file_name, index=False, sep=";", encoding='utf-8-sig')
files.download(output_file_name)
print(f"\nEl archivo {output_file_name} ha sido descargado")

# Generar y exportar el archivo consolidado total de consumo por frontera
consolidated_total_df = consolidated_df.groupby(["CODIGO FRONTERA", "NOMBRE FRONTERA","TIPO DE AGRUPACIÓN", "IMPO - EXPO"], as_index=False).agg({"TOTAL CONSUMO": "sum"})
output_total_file_name = f"total_consumo_{mes_archivos}_{anio_actual}.csv"
consolidated_total_df.to_csv(output_total_file_name, index=False, sep=";", encoding='utf-8-sig')
files.download(output_total_file_name)
print(f"\nEl archivo {output_total_file_name} ha sido descargado")

## Borrar la carpeta `content`

Ejecuta el siguiente código para borrar rapidamente la carpeta "content" y todos sus archivos temporales

In [2]:
# Ruta de la carpeta predeterminada en Google Colab
folder_path = '/content'

# Eliminar todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)

print("\nTodos los archivos cargados han sido eliminados de la carpeta content de Google Colab.")


Todos los archivos cargados han sido eliminados de la carpeta content de Google Colab.


## Conexión al ftp

In [ ]:
!pip install shareplum # Ejecutar en una celda aparte
!pip show shareplum # Ejecutar en una celda aparte

from ftplib import FTP_TLS
from shareplum import Site
from shareplum import Office365
from shareplum.site import Version
import os
from datetime import datetime

# 🔹 DATOS DE CONEXIÓN FTP
ftp_server = "xmftps.xm.com.co"
ftp_port = 210
ftp_user = "ISAMDNT\\1098742265"
ftp_password = "Ru1t0qu309p2025*"

# 🔹 DATOS DE SHAREPOINT
sharepoint_url = "https://ruitoqueesp1.sharepoint.com"
sharepoint_site = "fronterascomerciales"  # Nombre del sitio en SharePoint
sharepoint_user = "rbadillo@ruitoqueesp.com"
sharepoint_password = "r2083502R"

# 🔹 VARIABLES PARA AÑO Y MES (MODIFICABLES)
anio_especifico = None  # Cambia esto si quieres un año específico (ej: 2023)
mes_especifico = None   # Cambia esto si quieres un mes específico (ej: 02 para febrero)

# 🔹 OBTENER AÑO Y MES SEGÚN CONFIGURACIÓN
anio_actual = anio_especifico if anio_especifico else datetime.now().year
numero_mes = f"{mes_especifico:02d}" if mes_especifico else datetime.now().strftime("%m")
ruta_ftp = f"/INFORMACION_XM/USUARIOSK/RTQC/sic/comercia/{anio_actual}-{numero_mes}"

# 🔹 DEFINIR FOLDER DESTINO DE SHAREPOINT
sharepoint_folder = f"Documentos Compartidos/aenc/{anio_actual}/{numero_mes}"

# 🔹 CONECTARSE A FTP
try:
    print("Conectando al servidor FTP...")
    connection = FTP_TLS()
    connection.connect(ftp_server, ftp_port)
    connection.login(user=ftp_user, passwd=ftp_password)
    connection.set_pasv(True)
    connection.prot_p()
    print("Conexión establecida.")

    # Navegar a la carpeta en el FTP
    connection.cwd(ruta_ftp)
    print(f"Navegando a la ruta FTP: {ruta_ftp}")

    # Listar archivos y filtrar los que comienzan con "aenc" o "tfroc"
    archivos_ftp = connection.nlst()
    archivos_descargar = [archivo for archivo in archivos_ftp if archivo.startswith(("aenc", "tfroc"))]

    if not archivos_descargar:
        print("No se encontraron archivos para descargar.")
    else:
        print(f"Archivos a descargar: {archivos_descargar}")

        # Crear carpeta temporal para almacenar archivos descargados
        temp_folder = "archivos_descargados"
        os.makedirs(temp_folder, exist_ok=True)

        for archivo in archivos_descargar:
            local_path = os.path.join(temp_folder, archivo)
            with open(local_path, "wb") as f:
                connection.retrbinary(f"RETR {archivo}", f.write)
            print(f"Archivo descargado: {archivo}")

except Exception as e:
    print(f"Error en la conexión FTP: {e}")

finally:
    if 'connection' in locals():
        connection.quit()
        print("Conexión FTP cerrada.")

# 🔹 CONECTARSE A SHAREPOINT Y SUBIR ARCHIVOS
try:
    print("Conectando a SharePoint...")
    authcookie = Office365(sharepoint_url, username=sharepoint_user, password=sharepoint_password).GetCookies()
    site = Site(f"{sharepoint_url}/sites/{sharepoint_site}", authcookie=authcookie, version=Version.v365)
    folder = site.Folder(sharepoint_folder)
    print("Conexión a SharePoint establecida.")

    # Subir los archivos al SharePoint
    for archivo in archivos_descargar:
        local_path = os.path.join(temp_folder, archivo)
        with open(local_path, "rb") as file:
            file_content = file.read()
            folder.upload_file(file_content, archivo)
            print(f"Archivo subido a SharePoint: {archivo}")

except Exception as e:
    print(f"Error en la conexión a SharePoint: {e}")